# Desarrollo de Prueba - Ingeniería de datos 
### Christian Orrego

In [1]:
import pandas as pd
import math

### Carga de datos

In [2]:
df_barrios  = pd.read_csv("D:\Download\Prueba Tecnica\exa_trx_clientes\exa_barrios_cali.csv",sep=',')
df_dispositivos =  pd.read_csv("D:\Download\Prueba Tecnica\exa_trx_clientes\exa_dispositivos_cali.csv",sep=',')
df_trx_cli =   pd.read_csv("D:\Download\Prueba Tecnica\exa_trx_clientes\exa_trx_clientes.csv",sep=',')

In [3]:
#Exploracion inicial de cantidad de registros y estructura
#df_barrios.head()
#df_dispositivos.head()
#df_trx_cli.head()
df_barrios.count()
#df_dispositivos.count()
#df_trx_cli.count()

codigo    337
nombre    337
dtype: int64

In [4]:
#Renombrar columnas para permitir el merge
df_barrios.rename(columns={'codigo':'id_barrio'},inplace=True)
df_dispositivos.rename(columns = {'codigo':'cod_dispositivo','tipo':'canal'},inplace=True)

## Ejercicios a

In [5]:
# 
a = df_trx_cli.merge(df_dispositivos,on=['cod_dispositivo','canal']).groupby(['num_doc','tipo_doc']).id_barrio.nunique()
a =  a[a>=3]
a.count() # Se valida la cantidad de registros contra el SQL
#a.to_csv("D:\Download\Prueba Tecnica\exa_trx_clientes\prog_ej_a.csv")

4739

## Ejercicio b

In [6]:
#combinar trx cli con dispositivos
b =  df_trx_cli.merge(df_dispositivos,on=['cod_dispositivo','canal'])

# numero de transacciones por canal y cliente
b_sum = b.groupby(['num_doc','tipo_doc','canal'])["num_trx"].sum()

# numero de transaccionesp por cliente
tot_trx = b.groupby(['num_doc','tipo_doc'])["num_trx"].sum()

#combinacion de total y num trx por canal
com =  b_sum.to_frame().reset_index()
com = com.merge(tot_trx.to_frame(),on=['num_doc','tipo_doc'])
com.rename(columns={'num_trx_y':'tot_num_trx'},inplace=True)
com["porj"] = com["num_trx_x"]/com["tot_num_trx"]
com.sort_values(by=['num_doc','tipo_doc','porj'],ascending=[True,True,False],inplace=True)
#com
#com.to_csv("D:/Download/Prueba Tecnica/temp.csv")

canal_usados= []
client=''
por_tot = 0

#El daframe se recorre de forma ordenada y son seleccionados los canales por cliente hasta que el porcentaje total sea al menos a 0.51
for index, fila in com.iterrows():
    #print(fila)
    id_cli = str(fila['num_doc'])+str(fila['tipo_doc'])
    if client != id_cli:
        client = id_cli
        por_tot = fila['porj']
        canal_usados.append([fila['num_doc'],fila['tipo_doc'],fila['canal'],fila['porj']])
        #print([fila['num_doc'],fila['tipo_doc'],fila['canal'],fila['porj'],por_tot],)

    else:
        if por_tot < 0.51:
            canal_usados.append([fila['num_doc'],fila['tipo_doc'],fila['canal'],fila['porj']])
            #print([fila['num_doc'],fila['tipo_doc'],fila['canal'],fila['porj'],por_tot],)
        por_tot = por_tot +  fila['porj']
    #print(por_tot)
    
resultado = pd.DataFrame(canal_usados,columns=['num_doc','tipo_doc','canal','porj'])
#resultado
#resultado.to_csv("D:/Download/Prueba Tecnica/ej_d_python.csv")

In [7]:
#test
resultado[resultado["num_doc"]==1004793671201168182]

,num_doc,tipo_doc,canal,porj
6173,1004793671201168182,1,DISPENSADOR,0.705882


## Ejercicio c

In [8]:
c = df_trx_cli.merge(df_dispositivos,on=['cod_dispositivo','canal']).merge(df_barrios,on=['id_barrio'])
#crear id de cliente
c["id_cliente"] = c.apply(lambda x: str(x["num_doc"]) + str(x["tipo_doc"]), axis=1)
c = c.groupby(['id_barrio','nombre']).id_cliente.nunique().nlargest(10)
c
#c.to_csv("D:\Download\Prueba Tecnica\exa_trx_clientes\prog_ej_c.csv")

id_barrio  nombre                             
737998832  San Pedro                              2101
737998251  San Vicente                            1888
738000407  Urbanización San Juaquín               1756
737998194  Santa Mónica Residencial               1561
737998252  Chipichape                             1555
738000377  Unicentro Cali                         1498
737998124  Urbanización Militar                   1404
737999419  El Sena                                1341
737998095  Unidad Residencial Santiago de Cali    1327
737998087  El Cedro                               1256
Name: id_cliente, dtype: int64

## Ejercicios d

In [9]:
#Funcion para calcular la distancia entre dos puntos
def distancia(lat1,lon1,lat2,lon2):
    dist = math.sqrt((lat1 - lat2)**2 + (lon1 - lon2)**2)  
    return dist  

In [10]:
#Se encuentra el id del barrio
id_el_cedro = df_barrios.loc[df_barrios["nombre"] == "El Cedro"].id_barrio.values[0]
#filtra dispositivos del barrio
d = df_dispositivos.loc[df_dispositivos["id_barrio"] == id_el_cedro] 
comparacion = []
#Es calculada la diferencia entre los puntos y se almacena en un arreglo
for index_x, fila_x in d.iterrows():
    
    canal_x =  fila_x["canal"]
    cod_dispositivo_x = fila_x["cod_dispositivo"]
    
    lat1 = fila_x["latitud"]
    lon1 = fila_x["longitud"] 
    
    for index_y, fila_y in d.iterrows():
        
        
        canal_y =  fila_y["canal"]
        cod_dispositivo_y = fila_y["cod_dispositivo"]
        
        lat2 = fila_y["latitud"]
        lon2 = fila_y["longitud"] 
        
        dif = distancia(lat1,lon1,lat2,lon2)
        elemento = [canal_x,cod_dispositivo_x, lat1,lon1, canal_y,cod_dispositivo_y,lat2,lon2,dif]
        
        comparacion.append(elemento)
        
len(comparacion)

765625

In [11]:
df_comp = pd.DataFrame(comparacion, columns=['canal_x','cod_dispositivo_x', 'lat1','lon1', 'canal_y','cod_dispositivo_y','lat2','lon2','dif'])
#son seleccionados los 5 primeros registros para obtener los dispositivos más lejanos
df_comp.nlargest(5,['dif'])

,canal_x,cod_dispositivo_x,lat1,lon1,canal_y,cod_dispositivo_y,lat2,lon2,dif
369066,POS,12200283,3.436725,-76.538580,POS,4408003715,3.426447,-76.537534,0.010332
439941,POS,12626370,3.436725,-76.538580,POS,4408003715,3.426447,-76.537534,0.010332
509941,POS,13249230,3.436725,-76.538580,POS,4408003715,3.426447,-76.537534,0.010332
605046,POS,4408003715,3.426447,-76.537534,POS,12200283,3.436725,-76.538580,0.010332
605127,POS,4408003715,3.426447,-76.537534,POS,12626370,3.436725,-76.538580,0.010332
